### Fine-Tune a SLM for medical domain, which will predict if the user utterances are benign or harmless. Supervised fine-tunung was performed using PEFT-LORA.

Data For fune-tuning: [CARES](https://arxiv.org/pdf/2505.11413)

In [ ]:
!pip install -q transformers trl bitsandbytes peft

In [ ]:
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model

import pandas as pd
import re

USE_GPU = False

In [ ]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("HFXM/CARES-18K")

In [ ]:
display(ds)

In [ ]:
import pandas as pd

harmful_levels = pd.Series(ds['train']['harmful_level'])
display(harmful_levels.value_counts())

In [ ]:
def generate_responses(model, tokenizer, user_message, system_message=None,
                       max_new_tokens=100):
    # Format chat using tokenizer's chat template
    messages = []
    # Add a strong system message to force the output format
    system_message = "You are a helpful assistant. Your task is to classify text into exactly one of two categories: 'HARMFUL' or 'HARMLESS'. Do not provide any other text, explanations, or reasoning. Only output the category word."
    messages.append({"role": "system", "content": system_message})

    # We assume the data are all single-turn conversation
    messages.append({"role": "user", "content": user_message})

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)


    # Recommended to use vllm, sglang or TensorRT
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            # max_new_tokens=10, # Limiting max_new_tokens to encourage concise output
            # do_sample=False,
            # num_beams=1,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            # force_words_ids=force_words_ids,
            trust_remote_code=True # Add this line to resolve the error
        )
    input_len = inputs["input_ids"].shape[1]
    generated_ids = outputs[0][input_len:]
    response = tokenizer.decode(generated_ids, skip_special_tokens=True).strip().upper() # Convert to upper for consistent matching

    return response

def test_model_with_questions(model, tokenizer, questions,
                            system_message=None, title="Model Output"):
  print(f"\n=== {title} ===")
  responses= []
  for i, question in enumerate(questions, 1):
      response = generate_responses(model, tokenizer, question,
                                    system_message)
      # print(f"\nModel Input {i}:\n{question}\nModel Output {i}:\n{response}\n")


          # Update regex to search for 'harmless' or 'harmful'
      match = re.search(r'\b(HARMLESS|HARMFUL)\b', response)
      if match:
          predicted_category = match.group(1)
      else:
          predicted_category = response


      responses.append( predicted_category)
  return responses

def load_model_and_tokenizer(model_name, use_gpu = False):

    # Load base model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

    if use_gpu:
        model.to("cuda")

    if not tokenizer.chat_template:
        tokenizer.chat_template = """{% for message in messages %}\n  {% if message['role'] == 'system' %}System: {{ message['content'] }}\n\n                {% elif message['role'] == 'user' %}User: {{ message['content'] }}\n\n                {% elif message['role'] == 'assistant' %}Assistant: {{ message['content'] }} <|endoftext|>\n                {% endif %}\n                {% endfor %}"""

    # Tokenizer config
    if not tokenizer.pad_token:
        tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

def display_dataset(dataset):
    # Visualize the dataset
    rows = []
    for i in range(3):
        example = dataset[i]
        user_msg = next(m['content'] for m in example['messages']
                        if m['role'] == 'user')
        assistant_msg = next(m['content'] for m in example['messages']
                             if m['role'] == 'assistant')
        rows.append({
            'User Prompt': user_msg,
            'Assistant Response': assistant_msg
        })

    # Display as table
    df = pd.DataFrame(rows)
    pd.set_option('display.max_colwidth', None)  # Avoid truncating long strings
    display(df)

def categorize_harmful_level(level):
    if level in [0, 1]:
        return "HARMLESS"
    elif level in [2, 3]:
        return "HARMFUL"
    else:
        return "UNKNOWN"

def convert_to_chatml(example):

    return {
        "messages": [
            # {"role": "system", "content": "Classify the harm label of the following text as harmful or harmless DIRECTLY"},
            {"role": "user", "content": f"{example["base_prompt"]}\n\n INSTRUCTIONS: Based on the above context, classify the harm label of the following text as HARMFUL or HARMLESS:\n\nText: {example["prompt"]}"},
            {"role": "assistant", "content":  example["harm_binary"]}
        ]
    }
def convert_to_chatml_infer(example):
  return { "content": f"{example["base_prompt"]}\n\n INSTRUCTIONS: Based on the above context, classify the harm label of the following text as HARMFUL or HARMLESS:\n\nText: {example["prompt"]}"}



def convert_to_chatml_kto(example):
  if example["harm_binary"] == "HARMLESS":
    int_label = 0
  elif example["harm_binary"] == "HARMFUL":
    int_label =1
  else:
    int_label = -1

  return {"prompt" : [{"content" : example["base_prompt"], "role": "user"}]
          , "completion":[{"content": example["prompt"], "role":"assistant"}]
          , "label": int_label
        }


def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }

def cal_score(true_labels, harmful_predictions):

  # Convert true_labels to numerical format (0 for harmless, 1 for harmful, -1 for unknown)
  true_labels_numeric = [0 if label == "HARMLESS" else (1 if label == "HARMFUL" else -1) for label in true_labels]

    # Convert true_labels to numerical format (0 for harmless, 1 for harmful, -1 for unknown)
  harmful_predictions_numeric = [0 if label == "HARMLESS" else (1 if label == "HARMFUL" else -1) for label in harmful_predictions]

    # Filter out samples where either true label or prediction is -1 (unknown/unclear)
  filtered_true_labels = []
  filtered_predictions = []
  for true, pred in zip(true_labels_numeric, harmful_predictions_numeric):
      if true in [0, 1] and pred in [0, 1]: # Only include valid true labels and predictions
          filtered_true_labels.append(true)
          filtered_predictions.append(pred)

  # Calculate accuracy on the filtered subset
  if len(filtered_predictions) > 0:
      correct_predictions = sum(1 for true, pred in zip(filtered_true_labels, filtered_predictions) if true == pred)
      accuracy = correct_predictions / len(filtered_predictions)
      print(f"\nAccuracy on the filtered subset of {len(filtered_predictions)} samples: {accuracy:.2%}")
  else:
      print("\nNo valid samples (with known true labels and valid predictions) to calculate accuracy.")

  # f1 score
  from sklearn.metrics import f1_score
  if len(filtered_predictions) > 0:
      f1 = f1_score(filtered_true_labels, filtered_predictions, average='weighted')
      print(f"F1 Score on the filtered subset of {len(filtered_predictions)} samples: {f1:.2%}")
  else:
      print("No valid samples (with known true labels and valid predictions) to calculate F1 Score.")

## Prepare Data for LLM Inference



In [ ]:
train_data = ds['train']
test = ds['test']

# apply categorization to train and test datasets
train_harmful_levels = [categorize_harmful_level(level) for level in train_data['harmful_level']]
test_harmful_levels = [categorize_harmful_level(level) for level in test['harmful_level']]

# Add new column using add_column method
train_data = train_data.add_column("harm_binary", train_harmful_levels)
test = test.add_column("harm_binary", test_harmful_levels)

# sub-sample train_data for quicker processing during testing
train_subset_size = 1000
test_subset_size = 20
train_data = train_data.select(range(train_subset_size))
test = test.select(range(test_subset_size))

display(pd.Series(train_data["harm_binary"]).value_counts())
display(pd.Series(test["harm_binary"]).value_counts())

## Perform Harmful Label Prediction



In [ ]:
display(train_data)

In [ ]:
train_data_sft = train_data.map(convert_to_chatml
                            , remove_columns = ["principle_index", 'generation_model', 'harmful_level', 'method', 'base_prompt', 'prompt', "harm_binary"]
                            )

display(train_data_sft)

In [ ]:
train_data_sft[0]

In [ ]:
display_dataset(train_data_sft)

In [ ]:
test_data = test.map(convert_to_chatml_infer
                            , remove_columns = ["principle_index", 'generation_model', 'harmful_level', 'method', 'base_prompt', 'prompt', "harm_binary"]
                            )
display(test_data)

In [ ]:
model, tokenizer = load_model_and_tokenizer("Qwen/Qwen3-0.6B", USE_GPU)

In [ ]:
converted_list = list(test_data['content'])
print(converted_list)

In [ ]:
test_res = test_model_with_questions(model, tokenizer, converted_list,
                          title="Base Model (Before SFT) Output")
print (test_res)

In [ ]:
len(test_res)

In [ ]:
cal_score(test["harm_binary"], test_res)

In [ ]:
lora_config = LoraConfig(
    r=8, # Rank of the update matrices. Lower rank results in smaller models.
    lora_alpha=32, # Alpha parameter for LoRA scaling. A higher alpha value assigns more weight to the LoRA layers.
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"], # Target specific modules to apply LoRA. These are common layers in transformer models.
    lora_dropout=0.05, # Dropout probability for LoRA layers.
    bias="none", # Type of bias to be applied. "none" means no bias. Other options include "all" or "lora_only".
    task_type="CAUSAL_LM", # Task type, indicating that the model is for causal language modeling.
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

print("Model converted to PEFT model with LoRA configuration.")
model.print_trainable_parameters()

In [ ]:
display_dataset(train_data_sft)

In [ ]:
sft_config = SFTConfig(
    learning_rate=5e-5, # Learning rate for training.
    num_train_epochs=1, #  Set the number of epochs to train the model.
    per_device_train_batch_size=4, # Batch size for each device (e.g., GPU) during training.
    gradient_accumulation_steps=8, # Number of steps before performing a backward/update pass to accumulate gradients.
    gradient_checkpointing=True, # Enable gradient checkpointing to reduce memory usage during training at the cost of slower training speed.
    logging_steps=2,  # Frequency of logging training progress (log every 2 steps).
    report_to=["wandb"], # Integrate Weights & Biases logging.
    run_name="my-lora-sft-run",
    bf16=False, # Disable bf16
    fp16=True # Enable fp16
)

sft_trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_data_sft,
    processing_class=tokenizer,
)
sft_trainer.train()

# Evaluate on the test-set post-SFT

In [ ]:
output_dir = "./fine_tuned_model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, LoraConfig
import torch

output_dir = "./fine_tuned_model"

# Load the base model first
base_model_name = 'Qwen/Qwen3-0.6B' # Assuming this was the base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16, # Or torch.float16 if bfloat16 is not supported
    device_map='auto',
    trust_remote_code=True
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(output_dir)

# Load the LoRA adapter configuration
lora_config = LoraConfig.from_pretrained(output_dir)

# Apply the LoRA adapter to the base model
loaded_model = PeftModel.from_pretrained(base_model, output_dir)

print("Model and tokenizer loaded successfully!")

In [ ]:
# print(loaded_model)
# print(tokenizer)

In [ ]:
test_res_post = test_model_with_questions(loaded_model, tokenizer, converted_list,
                          title="Fine-tuned Model (Post SFT) Output")
print(test_res_post)

In [ ]:
len(converted_list), len(test_res_post)

In [ ]:
cal_score(test["harm_binary"], test_res_post)

### Experiment with KTO

In [ ]:
from trl.experimental.kto import KTOConfig, KTOTrainer

In [ ]:
train_data_kto = train_data.map(convert_to_chatml_kto
                            , remove_columns = ["principle_index", 'generation_model', 'harmful_level', 'method', 'base_prompt', 'prompt', "harm_binary"]
                            )

display(train_data_kto)

In [ ]:
train_data_kto[0]

In [ ]:
training_args_kto = KTOConfig(
                        learning_rate=5e-5, # Learning rate for training.
                        num_train_epochs=1, #  Set the number of epochs to train the model.
                        per_device_train_batch_size=4, # Batch size for each device (e.g., GPU) during training.
                        gradient_accumulation_steps=8, # Number of steps before performing a backward/update pass to accumulate gradients.
                        # gradient_checkpointing=True, # Enable gradient checkpointing to reduce memory usage during training at the cost of slower training speed.
                        logging_steps=2,  # Frequency of logging training progress (log every 2 steps).
                        report_to=["wandb"], # Integrate Weights & Biases logging.
                        run_name="my-lora-dpo-run")

trainer_kto = KTOTrainer(model=loaded_model
                     , args=training_args_kto
                     , processing_class=tokenizer
                     , train_dataset=train_data_kto)
trainer_kto.train()

In [ ]:
# save model
output_dir = "./fine_tuned_model_kto"
loaded_model.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")

In [ ]:
# load model
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, LoraConfig
import torch

output_dir = "./fine_tuned_model_kto"
load_model_kto = AutoModelForCausalLM.from_pretrained(
    output_dir,
    torch_dtype=torch.bfloat16, # Or torch.float16 if bfloat16 is not supported
    device_map='auto',
    trust_remote_code=True
)


In [ ]:
# display(load_model_kto)

In [ ]:
test_res_kto = test_model_with_questions(load_model_kto, tokenizer, converted_list,
                          title="Fine-tuned Model (Post KTO) Output")
print(test_res_kto)

In [ ]:
cal_score(test["harm_binary"], test_res_kto)

In [ ]:
## Future: test with Out-of-Domain data of Patient Safety:https://arxiv.org/abs/2507.07248
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds_ood = load_dataset("microsoft/PatientSafetyBench")
display(ds_ood)



## Insights
* **Base Model Performance**: The base model (Qwen/Qwen3-0.6B) showed poor performance in classifying harmful/harmless utterances with an accuracy of 50.00% and an F1 score of 38.10% on the test subset.
* **SFT Fine-tuning Impact**: Supervised Fine-Tuning (SFT) was crucial as the base model initially struggled to follow the specific instruction format (ChatML) and provide direct 'HARMFUL' or 'HARMLESS' classifications. SFT explicitly teaches the model to adhere to these instructions and output the desired format. After SFT, the model's classification capabilities significantly improved, with accuracy increasing to 60.00% and the F1 Score to 54.73%.
* **KTO Further Improvement**: Kahneman-Tversky Optimization (KTO) was applied on top of the SFT-tuned model to further refine its preferences and alignment. While SFT teaches the model how to respond, KTO helps in reinforcing desired responses (e.g., correctly classifying harmful content) and penalizing undesired ones. This further enhanced the model's performance, achieving the highest results among the tested models with an accuracy of 65.00% and an F1 Score of 61.73%.

This approach of using Supervised Fine-Tuning (SFT) followed by Kahneman-Tversky Optimization (KTO) with Parameter-Efficient Fine-Tuning (PEFT) like LoRA has several pros and cons:

### Pros:

* **Improved Performance and Alignment**: SFT helps the model learn specific instruction formats and desired output styles (e.g., classifying as 'HARMFUL' or 'HARMLESS'). KTO further refines the model's preferences, reinforcing correct classifications and penalizing incorrect ones, leading to higher accuracy and F1 scores.
* **Parameter Efficiency**: Using PEFT methods like LoRA significantly reduces the number of trainable parameters. This makes the fine-tuning process faster, less computationally expensive, and results in smaller adapter models that are easier to store and deploy.
* **Instruction Following**: The explicit SFT step is crucial for teaching the model to adhere to precise output formats, which is evident from the base model's initial struggle to follow the desired classification format.
* **Resource Friendly**: Fine-tuning a smaller base model (like Qwen3-0.6B) with PEFT makes the entire process more accessible and less demanding on hardware resources compared to full fine-tuning of large models.

### Cons:

* **Data Dependency**: The performance is highly dependent on the quality, size, and diversity of the fine-tuning dataset. If the dataset (CARES-18K) is not sufficiently representative, the model might not generalize well.
* **Hyperparameter Sensitivity**: Both SFT and KTO involve various hyperparameters (learning rate, batch size, etc.) that need careful tuning to achieve optimal results. Suboptimal choices can lead to poor performance.
Potential for Overfitting: While PEFT helps mitigate this, fine-tuning on a specific dataset, especially a smaller one, can still lead to overfitting, where the model performs well on the training/test set but poorly on unseen or out-of-domain data.
* **Limited Generalization (Out-of-Domain)**: Even with improvements on the in-domain test set, the model might still struggle with out-of-domain data (as highlighted in the 'Next Steps'), necessitating further evaluation and potential adaptation.
* **Computational Cost (Relative)**: While more efficient than full fine-tuning, the process still requires GPU resources and time, especially for larger datasets or more complex models.
* **Black Box Nature**: Understanding why the model makes a particular classification can still be challenging, which is a general limitation of deep learning models.

## Next Steps Section:

* **Out-of-Domain Data Testing**: Evaluate the fine-tuned models (SFT and KTO) on out-of-domain data like the Patient Safety dataset (microsoft/PatientSafetyBench) to assess generalization capabilities.
* **Explore Other Base Models**: Experiment with larger or different base models (e.g., Llama, Mistral) to observe performance variations.
* **Advanced PEFT Methods**: Investigate other Parameter-Efficient Fine-Tuning (PEFT) techniques beyond LoRA, such as QLoRA or Prompt Tuning.
* **Hyperparameter Tuning**: Conduct more extensive hyperparameter tuning for both SFT and KTO training to potentially achieve better performance.